In [1]:
%cd ~/ipython-notebook/dqn4wirelesscontrol/
import sys
import time
import pandas as pd
pd.set_option('mode.chained_assignment', None)  # block warnings due to DataFrame value assignment
# Project modules
from sleep_control.integration import Emulation
import sleep_control.traffic_emulator
import sleep_control.traffic_server
from sleep_control.controller import QController, DummyController
import qlearning.qtable
import qlearning.qnn
from qlearning.qtable import QAgent
from qlearning.qnn import QAgentNN

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


/home/admin-326/Data/ipython-notebook/dqn4wirelesscontrol


Setting-up data

In [2]:
session_df = pd.read_csv(filepath_or_buffer='./sleep_control/data/net_traffic_nonull.dat', 
                         sep=',', 
                         names=['uid','location','startTime_unix','duration_ms','domainProviders','domainTypes','domains','bytesByDomain','requestsByDomain'])
session_df.index.name = 'sessionID'
session_df['endTime_unix'] = session_df['startTime_unix'] + session_df['duration_ms']
session_df['startTime_datetime'] = pd.to_datetime(session_df['startTime_unix'], unit='ms')  # convert start time to readible date_time strings
session_df['endTime_datetime'] = pd.to_datetime(session_df['endTime_unix'], unit='ms')
session_df['totalBytes'] = session_df['bytesByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum bytes across domains
session_df['totalRequests'] = session_df['requestsByDomain'].apply(lambda x: x.split(';')).map(lambda x: sum(map(float, x)))  # sum requests across domains
session_df.sort(['startTime_datetime'], ascending=True, inplace=True)  # get it sorted
session_df['interArrivalDuration_datetime'] = session_df.groupby('location')['startTime_datetime'].diff()  # group-wise diff
session_df['interArrivalDuration_ms'] = session_df.groupby('location')['startTime_unix'].diff()  # group-wise diff

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Setting up emulation

In [3]:
# Setting up Emulation
te = sleep_control.traffic_emulator.TrafficEmulator(session_df=session_df, time_step=pd.Timedelta(seconds=2), verbose=2)
ts = sleep_control.traffic_server.TrafficServer(verbose=2)
# actions = [(s, c) for s in [True, False] for c in ['serve_all', 'queue_all', 'random_serve_and_queue']]
actions = [(True, None), (False, 'serve_all')]
# agent = QAgent(actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.1)
agent = QAgentNN(dim_state=(1, 1, 2), range_state=((((0, 10), (0, 10)),),),
                 learning_rate=0.01, reward_scaling=10, batch_size=100, freeze_period=50, memory_size=200, num_buffer=2,
                 actions=actions, alpha=0.5, gamma=0.5, explore_strategy='epsilon', epsilon=0.2,
                 verbose=2
                 )
c = QController(agent=agent)
emu = sleep_control.integration.Emulation(te=te, ts=ts, c=c)

New TrafficEmulator with parameters:
  head=2014-08-31 16:00:04.181000
  tail=2015-01-31 15:56:59.142000
  time_step=0 days 00:00:02
  epoch=0
  verbose=2


Start emulation

In [4]:
old_stdout = sys.stdout
log_file = open("message.log","w")
sys.stdout = log_file

while emu.epoch is not None:
    # log time
    print "Epoch {}, ".format(emu.epoch),
    left = emu.te.head_datetime + emu.te.epoch*emu.te.time_step
    right = left + emu.te.time_step
    print "{} - {}".format(left.strftime("%Y-%m-%d %H:%M:%S"), right.strftime("%Y-%m-%d %H:%M:%S"))
    emu.step()
    print

KeyboardInterrupt: 

In [5]:
sys.stdout = old_stdout
log_file.close()

In [9]:
reload(sleep_control.integration)
reload(sleep_control.traffic_server)
reload(sleep_control.traffic_emulator)
import qlearning
reload(qlearning.qnn)
reload(qlearning.qtable)

<module 'qlearning.qtable' from 'qlearning/qtable.pyc'>